In [98]:
import os
import yfinance as yf
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta

import yfinance as yf
import pandas as pd
import numpy as np
import streamlit as st
import os
from datetime import datetime, timedelta
from typing import List

import warnings
from pandas.errors import PerformanceWarning

warnings.filterwarnings('ignore', category=PerformanceWarning)


In [99]:
if not os.path.exists('./data'):
    os.makedirs('./data')

def download_stock_data(symbols: List[str], target_symbols: List[str], start_date: str, end_date: str):
    all_symbols = list(symbols + target_symbols)
    data = pd.DataFrame()
    remove_symbols = []

    # Download JPY=X data first
    jpy_data = yf.download("JPY=X", start=start_date, end=end_date, interval='1d')
    jpy_data = jpy_data["Adj Close"]

    for symbol in list(set(all_symbols)):
        file_name = f"./data/{symbol}.csv"
        if os.path.isfile(file_name):
            stock_data = pd.read_csv(file_name, index_col=0, parse_dates=True)
            stock_data.index = pd.to_datetime(stock_data.index)
            last_date = stock_data.index[-1]
            if last_date >= pd.Timestamp(datetime.now()):
                new_data = yf.download(symbol, start=last_date.date() + timedelta(days=1), end=end_date, interval='1d')
                if not new_data.empty and not pd.isna(new_data['Adj Close'].iloc[-1]):
                    # Convert US stocks to JPY
                    if symbol[-2:] != ".T" and symbol != "JPY=X":
                        stock_data = stock_data.mul(jpy_data, axis=0)
                    stock_data = stock_data.append(new_data)
                    stock_data = stock_data[~stock_data.index.duplicated(keep='last')]
                    with open(file_name, mode='w') as f:
                        stock_data.to_csv(f)
                else:
                    remove_symbols.append(symbol)
        else:
            stock_data = yf.download(symbol, start=start_date, end=end_date, interval='1d')
            if not stock_data.empty:
                # Convert US stocks to JPY
                if symbol[-2:] != ".T" and symbol != "JPY=X":
                    stock_data = stock_data.mul(jpy_data, axis=0)
                with open(file_name, mode='w') as f:
                    stock_data.to_csv(f)
            else:
                remove_symbols.append(symbol)

        try:
            data[symbol] = stock_data['Adj Close']
        except Exception as e:
            remove_symbols.append(symbol)
            pass

    for item in list(set(remove_symbols)):
        if item in symbols:
            symbols.remove(item)
        if item in target_symbols:
            target_symbols.remove(item)

    with open("./feat_symbols.txt", "w") as f:
        for symbol in symbols:
            f.write(f"{symbol}\n")
    with open("./target_symbols_all.txt", "w") as f:
        for symbol in target_symbols:
            f.write(f"{symbol}\n")

    return data, symbols, target_symbols


def prepare_data(data, symbols, target_symbols):
    prepared_data = {}
    for target_symbol in target_symbols:
        stock_data = pd.DataFrame(data[target_symbol])
        for symbol in symbols:
            if symbol != target_symbol:
                stock_data[symbol] = data[symbol]
        prepared_data[target_symbol] = stock_data
    
    for key in prepared_data.keys():

        # NoneをNaNに置換
        prepared_data[key] = prepared_data[key].replace(to_replace=['None', 'null', 'nan', 'NA'], value=np.nan)

        # 欠損値NaNを過去最新の値で埋める
        prepared_data[key] = prepared_data[key].fillna(method="ffill")
        prepared_data[key] = prepared_data[key].fillna(method="bfill")
        # prepared_data[key] = prepared_data[key].dropna(axis=1)            

        if prepared_data[key].isnull().any().any():
            print(f"{key} : NaN values found.")
    
    return prepared_data

In [100]:
# 銘柄リストを取得する関数
def get_symbol_list(file):
    with open(file, "r") as f:
        symbols = [line.strip() for line in f.readlines()]
    return symbols

In [101]:
def get_prepared_data(symbols, target_symbols, years):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years*365)
    data, symbols, target_symbols = download_stock_data(symbols, target_symbols, start_date, end_date)
    prepared_data = prepare_data(data, symbols, target_symbols)

    return prepared_data

In [102]:
years = 5 # 学習データの期間

symbols = get_symbol_list("feat_symbols.txt")
target_symbols = get_symbol_list("target_symbols_all.txt")

dict_prepared_data = get_prepared_data(symbols, target_symbols, years)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CBS: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BHGE: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- VAR: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- LB: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CTL: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BF.B: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed downloa

In [103]:
import pickle

with open("tests/prepared_data_all.pkl", "wb") as tf:
    pickle.dump(dict_prepared_data,tf)

with open("tests/prepared_data_all.pkl", "rb") as tf:
    dict_prepared_data = pickle.load(tf)
    
for key in dict_prepared_data.keys():
    display(dict_prepared_data[key])

,4584.T,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,1040.5,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,1120.0,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,1110.5,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,1082.5,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,1083.5,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,243.0,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,241.0,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,244.0,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825


,1557.T,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,29870.0,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,29970.0,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,30150.0,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,30350.0,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,30350.0,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,55640.0,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,55680.0,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,55600.0,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825


,8789.T,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,88.0,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,86.0,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,86.0,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,90.0,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,91.0,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,80.0,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,79.0,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,69.0,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825


,1893.T,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,629.252991,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,635.229614,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,640.352417,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,634.375854,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,637.791016,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,680.000000,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,681.000000,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,675.000000,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825


,7974.T,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,3552.361084,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,3607.345947,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,3539.195312,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,3446.262207,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,3431.547607,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,5730.000000,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,5725.000000,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,5631.000000,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825


...

,1360.T,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,2953.000000,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,2953.000000,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,2953.000000,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,2953.000000,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,2953.000000,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,727.599976,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,733.099976,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,732.599976,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825


,1579.T,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,21220.0,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,21220.0,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,21220.0,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,21220.0,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,21220.0,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,17305.0,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,17195.0,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,17185.0,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825


,MSFT,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,10156.712338,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,10060.954219,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,10122.620135,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,10257.691507,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,10256.239119,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,41467.410400,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,42251.793196,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,41581.719347,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825


,GOOG,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,5965.799234,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,5951.605817,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,5909.127548,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,5985.191382,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,5935.824604,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,14579.779739,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,15190.135844,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,15674.771161,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825


,IBM,JPY=X,1332.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-16,11941.475145,110.296997,510.773743,1109.780884,2573.569824,5226.751953,998.089661,1020.613403,1310.643066,1637.038452,...,14802.174757,2215.374201,20137.083448,4240.627200,2509.402683,7420.681690,8472.282335,11804.086078,5557.955235,8848.577263
2018-05-17,11937.448756,110.359001,517.112915,1102.357544,2558.009277,5226.751953,985.412659,1021.537903,1312.214722,1600.175659,...,15132.117538,2268.450562,19883.611737,4202.739976,2503.778259,7426.932940,8419.224310,11863.311406,5613.070622,8864.240381
2018-05-18,11953.338563,110.828003,518.018677,1132.875488,2557.145264,5260.857910,982.032166,1026.160522,1321.643677,1551.583862,...,15274.355071,2261.012131,19868.260009,4208.082332,2489.690223,7476.236287,8387.772767,11856.774998,5585.679093,9057.522373
2018-05-21,12075.979820,110.879997,516.207458,1136.999390,2525.158691,5226.751953,981.187134,1025.235718,1304.357056,1526.450195,...,15494.760561,2247.426565,20419.134521,4195.604790,2472.301971,7624.871892,8427.376362,11842.654035,5606.680747,9166.991422
2018-05-22,12053.420213,110.977997,517.112915,1125.039795,2510.462646,5235.278809,975.271118,1022.462585,1305.142822,1536.503784,...,15447.342620,2258.370950,20405.606711,4214.741485,2487.753093,7534.428221,8403.204100,11712.095476,5667.789779,9067.629313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-09,16366.794926,135.072998,601.000000,1469.000000,2628.000000,4775.000000,1163.000000,838.000000,1679.000000,1861.000000,...,18361.823684,3977.899896,15087.653470,9334.894977,1996.378875,14425.796604,18587.395344,18376.681796,3298.482623,24903.407990
2023-05-10,16507.840621,135.287994,596.000000,1458.000000,2646.000000,4780.000000,1172.000000,845.000000,1675.000000,1855.000000,...,18063.653588,3944.997896,15084.611374,9375.458424,1934.618346,14450.110350,18634.569090,18591.275941,3213.089867,25047.219198
2023-05-11,16211.119183,134.087006,587.000000,1476.000000,2661.000000,4730.000000,1089.000000,827.000000,1659.000000,1813.000000,...,17755.801038,3888.523163,14333.901105,9201.050694,1882.581554,14095.226399,18415.508860,18328.353125,3041.093328,24822.185825
